# Fine-tunning a model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

from transformers import (
  Trainer,TrainingArguments
)

from transformers import pipeline

from transformers import AutoModelForCausalLM, AutoProcessor

from PIL import Image

from sentence_transformers import SentenceTransformer

from sentence_transformers import util

## Preparing a dataset

In [ ]:
dataset = load_dataset("wikimedia/wikipedia", "20231101.en", split="train")
dataset = dataset.select(range(3))

model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Use tokenizer on text
dataset = dataset.map(lambda row: tokenizer(row["text"], padding=True, max_length=512, truncation=True), keep_in_memory=True)

## Building the trainer

In [ ]:
dataset = load_dataset("yelp_review_full")

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

model_name = "google-bert/bert-base-cased"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

training_args = TrainingArguments(output_dir=".results")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset
)

trainer.train()

## Using the fine-tunned model

In [ ]:
text_example = "I am a HUGE fan of romantic comedies."

# Create the classifier
classifier = pipeline(task="sentiment-analysis", model=".results")

# Classify the text
results = classifier(text=text_example)

print(results)

# Text generation

## Generating text from a text prompt

In [ ]:
# Set model name
model_name = "gpt2"

# Get the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

prompt = "Wear sunglasses when its sunny because"

# Tokenize the input
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate the text output
output = model.generate(input_ids, num_return_sequences=1)

# Decode the output
generated_text = tokenizer.decode(output[0])

print(generated_text)

## Generating a caption from an image

In [ ]:
image = Image.open("images/profile.jpeg")

# Get the processor and model
processor = AutoProcessor.from_pretrained("microsoft/git-base-coco")
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-coco")

# Process the image
pixels = processor(images=image, return_tensors="pt").pixel_values

# Generate the ids
output = model.generate(pixel_values=pixels)

# Decode the output
caption = processor.batch_decode(output)

print(caption[0])

# Embeddings

## Generate embeddings for a sentence

In [ ]:
sentence = "Programmers, do you put your comments (before|after) the related code?"

# Create the first embedding model
embedder1 = SentenceTransformer("all-MiniLM-L6-v2")

# Embed the sentence
embedding1 = embedder1.encode([sentence])

# Create and use second embedding model
embedder2 = SentenceTransformer("sentence-transformers/paraphrase-albert-small-v2")
embedding2 = embedder2.encode([sentence])
 
# Compare the shapes
print(embedding1.shape == embedding2.shape)

# Semantic search

In [ ]:
sentences = [
    'Programmers, do you put your comments (before|after) the related code?', 
    'How sure are we that there were never any intelligent dinosaurs?', 
    'Can anyone suggest a desktop book reader for Mac that works similar to Stanza on the iPhone?', 
    'I will be in Lima, Ohio Monday night/tuesday on business. What is there to do, and see in the area?', 
    "I'm looking for a good quality headset that doesn't cost too much. Any recommendations?", 
    'How do I get a list of all the duplicate items using LINQ?', 
    "Please help me figure out why it's so tough for me to connect to Valve games. It's driving me insane.", 
    "Is there such a thing as 'good' instant coffee?", 
    'How do I get the distinct/unique values in a column in Excel?'
]

embedder = SentenceTransformer("all-MiniLM-L6-v2")

sentence_embeddings = embedder.encode(sentences)

query = "I need a desktop book reader for Mac"

# Generate embeddings
query_embedding = embedder.encode([query])[0]

# Compare embeddings
hits = util.semantic_search(query_embedding, sentence_embeddings, top_k=2)

# Print the top results
for hit in hits[0]:
    print(sentences[hit["corpus_id"]], "(Score: {:.4f})".format(hit["score"]))